In [5]:
import torch.nn.functional as F
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as transforms
import copy
from utils import set_seed, load_backbone_state_dict_only
from utils import plotSamples
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import sys
from utils import set_seed
import torch
from tqdm.auto import tqdm
from scipy.stats import invgamma
from copy import deepcopy
from IGML import ML_IG as estimator
from Sampler import Sampler
from test import evaluate, get_model_with_ml
from dataset import Dataset
import sys
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'


1.7.1


In [6]:
from model.ProtoNet import ProtoNet
from model.Bayesian import Bayesian
from model.MAP import MAP
from model.PGLR import PGLR
from model.LR import LR

In [7]:




dataset_name = 'M'
models_paths = {}
models_paths['O'] = {}
models_paths['O']["Scratch"]    = "../experiments/omniglot/omniglot_random/check_point"
models_paths['O']["Pretrained"] = "../experiments/omniglot/omniglot_Pretrain_final/check_point"
models_paths['O'][LR]           = "../experiments/omniglot/omniglot_MTLR_final/check_point"
models_paths['O'][PGLR]         = "../experiments/omniglot/omniglot_PGLR_final/check_point"
models_paths['O'][ProtoNet]      = "../experiments/omniglot/omniglot_Protonet_final/check_point"
models_paths['O'][MAP]          = "../experiments/omniglot/omniglot_MAP_final/check_point"
models_paths['O'][Bayesian]     = "../experiments/omniglot/omniglot_Bayesian_final/check_point"


models_paths['M'] = {}
models_paths['M']["Scratch"]    = "../experiments/miniimagenet/imagenet_random/check_point"
models_paths['M']["Pretrained"] = "../experiments/miniimagenet/imagenet_pretrain_final/check_point"
models_paths['M'][LR]           = "../experiments/miniimagenet/imagenet_MTLR_final/check_point"
models_paths['M'][PGLR]         = "../experiments/miniimagenet/imagenet_PGLR_final/check_point"
models_paths['M'][ProtoNet]      = "../experiments/miniimagenet/imagenet_protonet_final/check_point"
models_paths['M'][MAP]          = "../experiments/miniimagenet/imagenet_map_final/check_point"
models_paths['M'][Bayesian]     = "../experiments/miniimagenet/imagenet_bayesian_final/check_point"

lrs = {}
lrs['M'] = {}
lrs['M']['not_iid'] = {}
lrs['M']['not_iid'][PGLR] = {}


lrs['M']['not_iid'][PGLR]["Pretrained"] = 10.0
lrs['M']['not_iid'][PGLR][ProtoNet]     = 1000.0
lrs['M']['not_iid'][PGLR][Bayesian]     = 1000.0
lrs['M']['not_iid'][PGLR][MAP]          = 1000.0
lrs['M']['not_iid'][PGLR][PGLR]         =  1.0
lrs['M']['not_iid'][PGLR][LR]           = 10.0
lrs['M']['not_iid'][PGLR]['Scratch']    = 1.0

lrs['M']['not_iid'][LR] = {}


lrs['M']['not_iid'][LR]["Pretrained"] = 1e-5
lrs['M']['not_iid'][LR][ProtoNet]     = 1e-6
lrs['M']['not_iid'][LR][Bayesian]     = 1e-6
lrs['M']['not_iid'][LR][MAP]          = 1e-6
lrs['M']['not_iid'][LR][PGLR]         = 1e-6
lrs['M']['not_iid'][LR][LR]           = 1e-5
lrs['M']['not_iid'][LR]['Scratch']    = 1e-6

lrs['O'] = {}
lrs['O']['not_iid'] = {}
lrs['O']['not_iid'][PGLR] = {}


lrs['O']['not_iid'][PGLR]["Pretrained"] =   1.0
lrs['O']['not_iid'][PGLR][ProtoNet]     =  10.0
lrs['O']['not_iid'][PGLR][Bayesian]     = 100.0
lrs['O']['not_iid'][PGLR][MAP]          = 100.0
lrs['O']['not_iid'][PGLR][PGLR]         =   0.1
lrs['O']['not_iid'][PGLR][LR]           =   1.0
lrs['O']['not_iid'][PGLR]['Scratch']    =   10.0

lrs['O']['not_iid'][LR] = {}


lrs['O']['not_iid'][LR]["Pretrained"] = 1e-5
lrs['O']['not_iid'][LR][ProtoNet]     = 1e-6
lrs['O']['not_iid'][LR][Bayesian]     = 1e-5
lrs['O']['not_iid'][LR][MAP]          = 1e-5
lrs['O']['not_iid'][LR][PGLR]         = 1e-6
lrs['O']['not_iid'][LR][LR]           = 1e-4
lrs['O']['not_iid'][LR]['Scratch']    = 1e-6

lrs['M']['iid'] = {}
lrs['M']['iid'][PGLR] = {}
lrs['M']['iid'][PGLR]["Pretrained"]  = lrs['M']['iid'][PGLR]["Scratch"] = 1.0
lrs['M']['iid'][PGLR][PGLR] = 0.1
lrs['M']['iid'][LR] = {}
lrs['M']['iid'][LR]["Pretrained"] = lrs['M']['iid'][LR]["Scratch"] =  1e-3
lrs['M']['iid'][LR][LR] = 1e-3


lrs['O']['iid'] = {}
lrs['O']['iid'][PGLR] = {}
lrs['O']['iid'][PGLR]["Pretrained"] = lrs['O']['iid'][PGLR]["Scratch"] = 10.0
lrs['O']['iid'][PGLR][PGLR] = 1.0

lrs['O']['iid'][LR] = {}
lrs['O']['iid'][LR]["Pretrained"] = lrs['O']['iid'][LR]["Scratch"] = 1e-3
lrs['O']['iid'][LR][LR] = 1e-3


Models_with_ML = {Bayesian, MAP}
Models_with_LR = {LR, PGLR}

In [8]:
from datasets.omniglot.TrainParams import MetaTrainParams as MetaTrainParams_OOO
from datasets.omniglot.TestParams  import MetaTestParams as MetaTestParams_OOO

from datasets.miniimagenet.TrainParams import MetaTrainParams as MetaTrainParams_MMM
from datasets.miniimagenet.TestParams  import MetaTestParams  as MetaTestParams_MMM

def get_params_sampler_datasets (dataset_name):


    if dataset_name == 'O':
        params_test = MetaTestParams_OOO()
        params_train = MetaTrainParams_OOO()
    else:
        params_test = MetaTestParams_MMM()
        params_train = MetaTrainParams_MMM()
        

    params_train.meta_train_transforms = params_train.transforms_for_test

    dataset_train = Dataset.get_dataset(params_train, params_train.meta_train_transforms)

    dataset_test  = Dataset.get_dataset(params_test,  transform=params_test.meta_transforms)

    if dataset_name == "O":
        params_test.query_num_train_tasks = params_test.support_num_train_tasks = 600
        params_test.support_inner_step = 15
        params_test.query_train_inner_step = 5
        params_test.meta_test_steps = 100
    else:
        params_test.query_num_train_tasks = params_test.support_num_train_tasks = 20
        params_test.support_inner_step = 100
        params_test.query_train_inner_step = 100
        params_test.meta_test_steps = 100

    return params_train, params_test, dataset_train, dataset_test



In [9]:
params_train, params_test, dataset_train, dataset_test = get_params_sampler_datasets(dataset_name)

../datasets/miniimagenet/ train
Total classes =  64
../datasets/miniimagenet/ test
Total classes =  20


In [10]:
Source_Backbones   = ["Scratch", "Pretrained", LR, PGLR, ProtoNet, MAP, Bayesian]
Destination_Models = [LR, PGLR, ProtoNet, MAP, Bayesian]

In [17]:
Source_Backbones   = [Bayesian]
Destination_Models = [LR]

In [ ]:
results = {}
for Source_Backbone in tqdm (Source_Backbones, "source backbone"):
    results[Source_Backbone] = {}
    for Destination_Model in Destination_Models:
        print (f"Source Backbone:{Source_Backbone} , Destination_Model:{Destination_Model}")

        if Destination_Model in Models_with_LR:
            lr = lrs[dataset_name]['not_iid'][Destination_Model][Source_Backbone]
            print (f"using lr{lr}")
            params_train.inner_lr = lr
        
        model = Destination_Model(params_train).to(device)
        
        model.back_bone.load_state_dict (torch.load (models_paths[dataset_name][Source_Backbone]))
        
        model = get_model_with_ml(model, params_train, dataset_train)

        acc_mean, acc_std = evaluate(params_test, model, dataset_test, verbose=False)
        print (acc_mean, acc_std)
            
        results[Source_Backbone][Destination_Model] = acc_mean, acc_std
        
    

In [11]:
for key in results.keys():
    for key2 in results[key].keys():
        print (f"{key},{key2}:", results[key][key2])

Scratch,<class 'model.LR.LR'>: (0.0044466666666666665, 0.0009276254033223157)
Scratch,<class 'model.PGLR.PGLR'>: (0.3271066666666669, 0.008257371117842412)
Scratch,<class 'model.ProtoNet.ProtoNet'>: (0.36240000000000044, 0.007625687582842032)
Scratch,<class 'model.MAP.MAP'>: (0.38086666666666713, 0.007724707833381795)
Scratch,<class 'model.Bayesian.Bayesian'>: (0.3761666666666671, 0.0077155686763840745)
Pretrained,<class 'model.LR.LR'>: (0.25712, 0.006519291031665652)
Pretrained,<class 'model.PGLR.PGLR'>: (0.6268666666666673, 0.008262095510354594)
Pretrained,<class 'model.ProtoNet.ProtoNet'>: (0.7395000000000006, 0.00661253019316775)
Pretrained,<class 'model.MAP.MAP'>: (0.7630800000000009, 0.00811433847512471)
Pretrained,<class 'model.Bayesian.Bayesian'>: (0.7584133333333343, 0.00805413903254442)
<class 'model.LR.LR'>,<class 'model.LR.LR'>: (0.6389600000000006, 0.006858454636432297)
<class 'model.LR.LR'>,<class 'model.PGLR.PGLR'>: (0.7463600000000012, 0.005334099944903806)
<class 'mode

In [8]:
for key in results.keys():
    for key2 in results[key].keys():
        print (f"{key},{key2}:", results[key][key2])

Scratch,<class 'model.LR.LR'>: (0.05537000000000001, 0.008713386253346054)
Scratch,<class 'model.PGLR.PGLR'>: (0.153375, 0.01234065942322371)
Scratch,<class 'model.ProtoNet.ProtoNet'>: (0.1782, 0.007603946343840153)
Scratch,<class 'model.MAP.MAP'>: (0.18903999999999999, 0.008182200193102097)
Scratch,<class 'model.Bayesian.Bayesian'>: (0.18893000000000001, 0.008245610953713496)
Pretrained,<class 'model.LR.LR'>: (0.32811999999999997, 0.03127268136888809)
Pretrained,<class 'model.PGLR.PGLR'>: (0.5419400000000002, 0.009918487787964444)
Pretrained,<class 'model.ProtoNet.ProtoNet'>: (0.5556350000000001, 0.010868039151567302)
Pretrained,<class 'model.MAP.MAP'>: (0.5591700000000001, 0.010623375169878928)
Pretrained,<class 'model.Bayesian.Bayesian'>: (0.5588500000000001, 0.010683281331126704)
<class 'model.LR.LR'>,<class 'model.LR.LR'>: (0.31112500000000004, 0.010676931909495355)
<class 'model.LR.LR'>,<class 'model.PGLR.PGLR'>: (0.4344250000000001, 0.010727155960458494)
<class 'model.LR.LR'>,<c

In [ ]:
Destination_Models = [LR, PGLR]

results = {}
for IID_LR in ["iid","not_iid"]:
    results[IID_LR] = {}
    for IID_test in ["iid_test","not_iid_test"]:
        results[IID_LR][IID_test] = {}
        for train_type in ["Pretrained","meta"]:
            results[IID_LR][IID_test][train_type] = {}
            for Destination_Model in tqdm (Destination_Models, "destination models"):
                print (f"IID_lr:{IID_LR} train_type:{train_type}, IID_test:{IID_test} , Destination_Model:{Destination_Model}")
                
                if train_type == "meta":
                    Source_Backbone = Destination_Model
                else:
                    Source_Backbone = train_type
                
                if Destination_Model in Models_with_LR:
                    lr = lrs[dataset_name][IID_LR][Destination_Model][Source_Backbone]
                    print (f"using lr{lr}")
                    params_train.inner_lr = lr

                model = Destination_Model(params_train).to(device)
                model.back_bone.load_state_dict (torch.load (models_paths[dataset_name][Source_Backbone]))
                model = get_model_with_ml(model, params_train, dataset_train)
                
                if IID_test == "iid_test":
                    IID_epochs = 5
                else:
                    IID_epochs = 0

                if IID_test == "iid_test" and (Destination_Model in [Bayesian, ProtoNet, MAP]):
                    continue
                if IID_test == "iid" and (Destination_Model in [Bayesian, ProtoNet, MAP]):
                    continue
                
                acc_mean, acc_std = evaluate(params_test, model, dataset_test, IID_epochs = IID_epochs,verbose=False)
                print (acc_mean, acc_std)

                results[IID_LR][IID_test][train_type][Destination_Model] = acc_mean, acc_std
            
        

In [7]:
results

{'iid': {'iid_test': {'Pretrained': {model.LR.LR: (0.7783666666666679,
     0.007045960229489979),
    model.PGLR.PGLR: (0.7459400000000009, 0.00760137122132882)},
   'meta': {model.LR.LR: (0.8400800000000019, 0.006373054387201152),
    model.PGLR.PGLR: (0.8684266666666693, 0.004898090330822014)}},
  'not_iid_test': {'Pretrained': {model.LR.LR: (0.048699999999999986,
     0.010356586739311786),
    model.PGLR.PGLR: (0.6826400000000007, 0.006666946660786937)},
   'meta': {model.LR.LR: (0.4662199999999994, 0.010636333954892514),
    model.PGLR.PGLR: (0.7908400000000028, 0.006117094444623688)}}},
 'not_iid': {'iid_test': {'Pretrained': {model.LR.LR: (0.40472000000000036,
     0.007752092190714316),
    model.PGLR.PGLR: (0.7417866666666673, 0.006206004261107938)},
   'meta': {model.LR.LR: (0.7451133333333345, 0.005686869867412927),
    model.PGLR.PGLR: (0.8641000000000024, 0.004618922433257257)}},
  'not_iid_test': {'Pretrained': {model.LR.LR: (0.25712, 0.006519291031665652),
    model.PGL

In [8]:
results

{'iid': {'iid_test': {'Pretrained': {model.LR.LR: (0.6095700000000001,
     0.009812242353305398),
    model.PGLR.PGLR: (0.56184, 0.010349125566925943)},
   'meta': {model.LR.LR: (0.48890000000000006, 0.011025425161870163),
    model.PGLR.PGLR: (0.57363, 0.0100308573910708)}},
  'not_iid_test': {'Pretrained': {model.LR.LR: (0.07098, 0.019289105733548147),
    model.PGLR.PGLR: (0.53263, 0.00979428915235811)},
   'meta': {model.LR.LR: (0.22328, 0.014263646097684842),
    model.PGLR.PGLR: (0.51403, 0.013859080056049909)}}},
 'not_iid': {'iid_test': {'Pretrained': {model.LR.LR: (0.3845300000000001,
     0.014307833518740718),
    model.PGLR.PGLR: (0.5598200000000001, 0.010279474694749723)},
   'meta': {model.LR.LR: (0.31499000000000005, 0.010583236744966074),
    model.PGLR.PGLR: (0.57012, 0.009944123893033511)}},
  'not_iid_test': {'Pretrained': {model.LR.LR: (0.32345, 0.028813408337091954),
    model.PGLR.PGLR: (0.54132, 0.008282366811485696)},
   'meta': {model.LR.LR: (0.30989, 0.010856